In [7]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime
import xgboost as xgb
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
from sklearn.model_selection import train_test_split
import math
import time

import os
print(os.listdir("../input"))
# Any results you write to the current directory are saved as output.

train = pd.read_csv('../input/train_dates_ordered.csv')
test = pd.read_csv('../input/test.csv')
sample = pd.read_csv('../input/sample_submission.csv')
train['date'] = pd.to_datetime(train['date'])
test['date'] = pd.to_datetime(test['date'])

cal = calendar()
holidays = cal.holidays(start=train['date'].min(), end=test['date'].max())

train['week_of_year']  = train['date'].dt.weekofyear
train['weekday']  = train['date'].dt.dayofweek
train['holiday'] = train['date'].isin(holidays)*1
train['month'] = train['date'].dt.month
train['day'] = train['date'].dt.dayofweek
train['year'] = train['date'].dt.year

test['month'] = test['date'].dt.month
test['day'] = test['date'].dt.dayofweek
test['year'] = test['date'].dt.year
test['week_of_year']  = test['date'].dt.weekofyear
test['weekday']  = test['date'].dt.dayofweek
test['holiday'] = test['date'].isin(holidays)*1

col = [i for i in test.columns if i not in ['date','id']]

['sample_submission.csv', 'train_dates_ordered.csv', 'train.csv', 'test.csv']


In [ ]:
def XGB_regressor(train_X, train_y, test_X, test_y, feature_names=None, seed_val=2017, num_rounds=1600):
    param = {}
    param['objective'] = 'reg:linear'
    param['eta'] = 0.03
    #param['gamma'] = 0.1885914265710256
    param['max_depth'] = 8
    param['silent'] = 1
    param['eval_metric'] = 'mae'
    param['min_child_weight'] = 1
    param['subsample'] = 0.5947
    param['colsample_bytree'] = 0.66123
    param['seed'] = seed_val
    num_rounds = num_rounds

    plst = list(param.items())

    xgtrain = xgb.DMatrix(train_X, label=train_y)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y)
        watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=20)
    else:
        xgtest = xgb.DMatrix(test_X)
        model = xgb.train(plst, xgtrain, num_rounds)
        
    return model

## Ordered train_test_split: 80% training

In [9]:
entries_per_day = 500
days_per_month = 30

total_months = math.floor((train.shape[0]/entries_per_day)/days_per_month)
training_ratio = 0.8
data_points = math.floor(total_months*500*30.4333*training_ratio)
print (data_points)

train_x = train.loc[:data_points][col]
y = train.loc[:data_points]['sales'] 
train_cv = train.loc[data_points+1:][col]
y_cv = train.loc[data_points+1:]['sales']

def XGB_regressor(train_X, train_y, test_X, test_y, feature_names=None, seed_val=2017, num_rounds=1600):
    param = {}
    param['objective'] = 'reg:linear'
    param['eta'] = 0.03
    #param['gamma'] = 0.1885914265710256
    param['max_depth'] = 8
    param['silent'] = 1
    param['eval_metric'] = 'mae'
    param['min_child_weight'] = 1
    param['subsample'] = 0.5947
    param['colsample_bytree'] = 0.66123
    param['seed'] = seed_val
    num_rounds = num_rounds

    plst = list(param.items())

    xgtrain = xgb.DMatrix(train_X, label=train_y)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y)
        watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=20)
    else:
        xgtest = xgb.DMatrix(test_X)
        model = xgb.train(plst, xgtrain, num_rounds)
        
    return model

start = time.time()
model = XGB_regressor(train_X = train_x, train_y = y, test_X = train_cv, test_y = y_cv)
y_test = model.predict(xgb.DMatrix(test[col]), ntree_limit = model.best_ntree_limit)
end = time.time()

sample['sales'] = y_test
sample.to_csv('xgb_ordered_test_split.csv', index=False)
print (end - start)


730399
[0]	train-mae:48.609	test-mae:56.7788
Multiple eval metrics have been passed: 'test-mae' will be used for early stopping.

Will train until test-mae hasn't improved in 20 rounds.
[1]	train-mae:47.1522	test-mae:55.3147
[2]	train-mae:45.7373	test-mae:53.7421
[3]	train-mae:44.3657	test-mae:52.3433
[4]	train-mae:43.0357	test-mae:51.0122
[5]	train-mae:41.7453	test-mae:49.6219
[6]	train-mae:40.4964	test-mae:48.3664
[7]	train-mae:39.2874	test-mae:47.1347
[8]	train-mae:38.116	test-mae:45.9587
[9]	train-mae:36.9825	test-mae:44.7279
[10]	train-mae:35.8897	test-mae:43.6248
[11]	train-mae:34.8357	test-mae:42.5588
[12]	train-mae:33.8224	test-mae:41.5281
[13]	train-mae:32.8387	test-mae:40.4305
[14]	train-mae:31.8989	test-mae:39.469
[15]	train-mae:30.9961	test-mae:38.5235
[16]	train-mae:30.1254	test-mae:37.5921
[17]	train-mae:29.2894	test-mae:36.6352
[18]	train-mae:28.4985	test-mae:35.7866
[19]	train-mae:27.7354	test-mae:34.9871
[20]	train-mae:27.0157	test-mae:34.0804
[21]	train-mae:26.3211	te

[200]	train-mae:7.24889	test-mae:8.86833
[201]	train-mae:7.24599	test-mae:8.86141
[202]	train-mae:7.22982	test-mae:8.84099
[203]	train-mae:7.21622	test-mae:8.81961
[204]	train-mae:7.21452	test-mae:8.81501
[205]	train-mae:7.19639	test-mae:8.79025
[206]	train-mae:7.18239	test-mae:8.77088
[207]	train-mae:7.16972	test-mae:8.75046
[208]	train-mae:7.1261	test-mae:8.69924
[209]	train-mae:7.10195	test-mae:8.66748
[210]	train-mae:7.07344	test-mae:8.63326
[211]	train-mae:7.07073	test-mae:8.62799
[212]	train-mae:7.06818	test-mae:8.62198
[213]	train-mae:7.05511	test-mae:8.60389
[214]	train-mae:7.0213	test-mae:8.55879
[215]	train-mae:7.01901	test-mae:8.5528
[216]	train-mae:7.00805	test-mae:8.53529
[217]	train-mae:6.99953	test-mae:8.52333
[218]	train-mae:6.96173	test-mae:8.47938
[219]	train-mae:6.94971	test-mae:8.45861
[220]	train-mae:6.94741	test-mae:8.45417
[221]	train-mae:6.9359	test-mae:8.43432
[222]	train-mae:6.91147	test-mae:8.40545
[223]	train-mae:6.88181	test-mae:8.36999
[224]	train-mae:6.87

[401]	train-mae:5.90395	test-mae:6.90463
[402]	train-mae:5.90388	test-mae:6.90465
[403]	train-mae:5.9006	test-mae:6.898
[404]	train-mae:5.90061	test-mae:6.89756
[405]	train-mae:5.90054	test-mae:6.89753
[406]	train-mae:5.8996	test-mae:6.89602
[407]	train-mae:5.89749	test-mae:6.89208
[408]	train-mae:5.89379	test-mae:6.88687
[409]	train-mae:5.89066	test-mae:6.88317
[410]	train-mae:5.88733	test-mae:6.87919
[411]	train-mae:5.88501	test-mae:6.87588
[412]	train-mae:5.88027	test-mae:6.87007
[413]	train-mae:5.88019	test-mae:6.86968
[414]	train-mae:5.8802	test-mae:6.86894
[415]	train-mae:5.88015	test-mae:6.86898
[416]	train-mae:5.87715	test-mae:6.86458
[417]	train-mae:5.87708	test-mae:6.86394
[418]	train-mae:5.87253	test-mae:6.85822
[419]	train-mae:5.86915	test-mae:6.851
[420]	train-mae:5.86616	test-mae:6.84638
[421]	train-mae:5.86429	test-mae:6.84351
[422]	train-mae:5.86247	test-mae:6.8412
[423]	train-mae:5.86246	test-mae:6.84121
[424]	train-mae:5.86247	test-mae:6.84121
[425]	train-mae:5.8583	t

[602]	train-mae:5.63866	test-mae:6.49644
[603]	train-mae:5.63863	test-mae:6.49637
[604]	train-mae:5.63635	test-mae:6.49294
[605]	train-mae:5.63583	test-mae:6.49142
[606]	train-mae:5.63363	test-mae:6.48831
[607]	train-mae:5.63136	test-mae:6.48564
[608]	train-mae:5.63061	test-mae:6.48415
[609]	train-mae:5.63062	test-mae:6.48415
[610]	train-mae:5.62776	test-mae:6.48056
[611]	train-mae:5.62727	test-mae:6.47904
[612]	train-mae:5.62542	test-mae:6.47675
[613]	train-mae:5.62421	test-mae:6.47464
[614]	train-mae:5.62196	test-mae:6.47187
[615]	train-mae:5.62071	test-mae:6.47038
[616]	train-mae:5.62072	test-mae:6.47023
[617]	train-mae:5.61933	test-mae:6.46873
[618]	train-mae:5.61812	test-mae:6.46674
[619]	train-mae:5.61656	test-mae:6.46477
[620]	train-mae:5.61658	test-mae:6.46473
[621]	train-mae:5.61617	test-mae:6.4642
[622]	train-mae:5.61616	test-mae:6.46373
[623]	train-mae:5.61614	test-mae:6.4636
[624]	train-mae:5.61444	test-mae:6.46154
[625]	train-mae:5.61369	test-mae:6.46013
[626]	train-mae:5.

[803]	train-mae:5.52814	test-mae:6.34488
[804]	train-mae:5.5275	test-mae:6.34401
[805]	train-mae:5.52744	test-mae:6.34409
[806]	train-mae:5.52712	test-mae:6.3435
[807]	train-mae:5.52637	test-mae:6.34236
[808]	train-mae:5.52588	test-mae:6.34175
[809]	train-mae:5.52499	test-mae:6.34079
[810]	train-mae:5.52497	test-mae:6.34087
[811]	train-mae:5.5247	test-mae:6.34065
[812]	train-mae:5.52437	test-mae:6.34029
[813]	train-mae:5.52325	test-mae:6.33892
[814]	train-mae:5.52325	test-mae:6.33892
[815]	train-mae:5.52325	test-mae:6.33888
[816]	train-mae:5.52267	test-mae:6.33827
[817]	train-mae:5.52266	test-mae:6.33823
[818]	train-mae:5.52218	test-mae:6.33747
[819]	train-mae:5.52218	test-mae:6.33752
[820]	train-mae:5.52115	test-mae:6.33599
[821]	train-mae:5.52112	test-mae:6.3361
[822]	train-mae:5.52101	test-mae:6.33566
[823]	train-mae:5.52042	test-mae:6.33481
[824]	train-mae:5.52022	test-mae:6.33453
[825]	train-mae:5.52017	test-mae:6.33444
[826]	train-mae:5.51945	test-mae:6.33328
[827]	train-mae:5.51

[1004]	train-mae:5.47083	test-mae:6.27339
[1005]	train-mae:5.47053	test-mae:6.27301
[1006]	train-mae:5.47052	test-mae:6.27308
[1007]	train-mae:5.47012	test-mae:6.27266
[1008]	train-mae:5.46974	test-mae:6.27209
[1009]	train-mae:5.46931	test-mae:6.27155
[1010]	train-mae:5.46929	test-mae:6.27138
[1011]	train-mae:5.46885	test-mae:6.27087
[1012]	train-mae:5.46884	test-mae:6.27092
[1013]	train-mae:5.46873	test-mae:6.27094
[1014]	train-mae:5.4686	test-mae:6.27065
[1015]	train-mae:5.46857	test-mae:6.27076
[1016]	train-mae:5.46856	test-mae:6.27086
[1017]	train-mae:5.46844	test-mae:6.27063
[1018]	train-mae:5.46831	test-mae:6.27062
[1019]	train-mae:5.46829	test-mae:6.27059
[1020]	train-mae:5.46827	test-mae:6.27063
[1021]	train-mae:5.46827	test-mae:6.27065
[1022]	train-mae:5.46804	test-mae:6.27024
[1023]	train-mae:5.46805	test-mae:6.27005
[1024]	train-mae:5.46762	test-mae:6.2697
[1025]	train-mae:5.46757	test-mae:6.26966
[1026]	train-mae:5.46755	test-mae:6.2697
[1027]	train-mae:5.46733	test-mae:6.2

[1201]	train-mae:5.44374	test-mae:6.24556
[1202]	train-mae:5.44372	test-mae:6.24573
[1203]	train-mae:5.44371	test-mae:6.24575
[1204]	train-mae:5.44361	test-mae:6.2455
[1205]	train-mae:5.44358	test-mae:6.24546
[1206]	train-mae:5.44345	test-mae:6.24547
[1207]	train-mae:5.44333	test-mae:6.24544
[1208]	train-mae:5.4433	test-mae:6.24542
[1209]	train-mae:5.4433	test-mae:6.24545
[1210]	train-mae:5.44328	test-mae:6.24546
[1211]	train-mae:5.44319	test-mae:6.24525
[1212]	train-mae:5.44318	test-mae:6.24526
[1213]	train-mae:5.44308	test-mae:6.24521
[1214]	train-mae:5.44294	test-mae:6.24515
[1215]	train-mae:5.44276	test-mae:6.24507
[1216]	train-mae:5.4426	test-mae:6.24472
[1217]	train-mae:5.44254	test-mae:6.24463
[1218]	train-mae:5.44243	test-mae:6.2445
[1219]	train-mae:5.44243	test-mae:6.24442
[1220]	train-mae:5.44229	test-mae:6.24436
[1221]	train-mae:5.44208	test-mae:6.2439
[1222]	train-mae:5.44191	test-mae:6.24386
[1223]	train-mae:5.4419	test-mae:6.24389
[1224]	train-mae:5.4419	test-mae:6.24384


[1398]	train-mae:5.42732	test-mae:6.23283
[1399]	train-mae:5.4273	test-mae:6.23282
[1400]	train-mae:5.4273	test-mae:6.23284
[1401]	train-mae:5.42728	test-mae:6.23289
[1402]	train-mae:5.42725	test-mae:6.23293
[1403]	train-mae:5.42714	test-mae:6.23253
[1404]	train-mae:5.42701	test-mae:6.23255
[1405]	train-mae:5.42693	test-mae:6.23259
[1406]	train-mae:5.42683	test-mae:6.23253
[1407]	train-mae:5.42681	test-mae:6.23245
[1408]	train-mae:5.4267	test-mae:6.23248
[1409]	train-mae:5.42659	test-mae:6.23221
[1410]	train-mae:5.42651	test-mae:6.23205
[1411]	train-mae:5.42648	test-mae:6.23204
[1412]	train-mae:5.42641	test-mae:6.23203
[1413]	train-mae:5.42631	test-mae:6.23193
[1414]	train-mae:5.42621	test-mae:6.23183
[1415]	train-mae:5.42614	test-mae:6.23169
[1416]	train-mae:5.42605	test-mae:6.2317
[1417]	train-mae:5.42591	test-mae:6.2317
[1418]	train-mae:5.42586	test-mae:6.23171
[1419]	train-mae:5.42577	test-mae:6.23165
[1420]	train-mae:5.42567	test-mae:6.23174
[1421]	train-mae:5.42559	test-mae:6.231

## Random train_test_split: 80% training

In [8]:
y = 'sales'
train_x, train_cv, y, y_cv = train_test_split(train[col],train[y], test_size=0.2, random_state=2018)

def XGB_regressor(train_X, train_y, test_X, test_y, feature_names=None, seed_val=2017, num_rounds=1600):
    param = {}
    param['objective'] = 'reg:linear'
    param['eta'] = 0.03
    #param['gamma'] = 0.1885914265710256
    param['max_depth'] = 8
    param['silent'] = 1
    param['eval_metric'] = 'mae'
    param['min_child_weight'] = 1
    param['subsample'] = 0.5947
    param['colsample_bytree'] = 0.66123
    param['seed'] = seed_val
    num_rounds = num_rounds

    plst = list(param.items())

    xgtrain = xgb.DMatrix(train_X, label=train_y)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y)
        watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=20)
    else:
        xgtest = xgb.DMatrix(test_X)
        model = xgb.train(plst, xgtrain, num_rounds)
        
    return model

start = time.time()
model = XGB_regressor(train_X = train_x, train_y = y, test_X = train_cv, test_y = y_cv)
y_test = model.predict(xgb.DMatrix(test[col]), ntree_limit = model.best_ntree_limit)
end = time.time()

sample['sales'] = y_test
sample.to_csv('xgb_random_test_split.csv', index=False)
print (end - start)

[0]	train-mae:50.2025	test-mae:50.1801
Multiple eval metrics have been passed: 'test-mae' will be used for early stopping.

Will train until test-mae hasn't improved in 20 rounds.
[1]	train-mae:48.697	test-mae:48.6762
[2]	train-mae:47.2368	test-mae:47.2171
[3]	train-mae:45.8195	test-mae:45.801
[4]	train-mae:44.4456	test-mae:44.428
[5]	train-mae:43.114	test-mae:43.0954
[6]	train-mae:41.8237	test-mae:41.8059
[7]	train-mae:40.573	test-mae:40.5562
[8]	train-mae:39.3629	test-mae:39.3456
[9]	train-mae:38.191	test-mae:38.1729
[10]	train-mae:37.0618	test-mae:37.0456
[11]	train-mae:35.9727	test-mae:35.9573
[12]	train-mae:34.9248	test-mae:34.9098
[13]	train-mae:33.9096	test-mae:33.8942
[14]	train-mae:32.9347	test-mae:32.9198
[15]	train-mae:32.0042	test-mae:31.9904
[16]	train-mae:31.1036	test-mae:31.09
[17]	train-mae:30.2381	test-mae:30.2254
[18]	train-mae:29.4132	test-mae:29.4007
[19]	train-mae:28.624	test-mae:28.6123
[20]	train-mae:27.8787	test-mae:27.8699
[21]	train-mae:27.1584	test-mae:27.151

[201]	train-mae:7.41312	test-mae:7.44272
[202]	train-mae:7.3868	test-mae:7.41648
[203]	train-mae:7.3725	test-mae:7.40208
[204]	train-mae:7.37066	test-mae:7.40022
[205]	train-mae:7.34622	test-mae:7.3758
[206]	train-mae:7.3319	test-mae:7.36153
[207]	train-mae:7.29599	test-mae:7.32598
[208]	train-mae:7.2822	test-mae:7.3122
[209]	train-mae:7.26855	test-mae:7.29856
[210]	train-mae:7.23564	test-mae:7.26618
[211]	train-mae:7.2328	test-mae:7.26342
[212]	train-mae:7.23015	test-mae:7.2608
[213]	train-mae:7.20802	test-mae:7.23866
[214]	train-mae:7.18641	test-mae:7.21747
[215]	train-mae:7.18388	test-mae:7.21492
[216]	train-mae:7.16925	test-mae:7.20035
[217]	train-mae:7.15351	test-mae:7.18445
[218]	train-mae:7.13218	test-mae:7.16321
[219]	train-mae:7.10349	test-mae:7.13485
[220]	train-mae:7.10128	test-mae:7.13256
[221]	train-mae:7.08861	test-mae:7.11988
[222]	train-mae:7.07725	test-mae:7.10873
[223]	train-mae:7.05995	test-mae:7.09138
[224]	train-mae:7.04201	test-mae:7.07363
[225]	train-mae:7.01503	

[402]	train-mae:5.98289	test-mae:6.0315
[403]	train-mae:5.98082	test-mae:6.02955
[404]	train-mae:5.9808	test-mae:6.02956
[405]	train-mae:5.98074	test-mae:6.02954
[406]	train-mae:5.97648	test-mae:6.02552
[407]	train-mae:5.97522	test-mae:6.0244
[408]	train-mae:5.97138	test-mae:6.02054
[409]	train-mae:5.96733	test-mae:6.01671
[410]	train-mae:5.96441	test-mae:6.01403
[411]	train-mae:5.95905	test-mae:6.00913
[412]	train-mae:5.95796	test-mae:6.00812
[413]	train-mae:5.95784	test-mae:6.00804
[414]	train-mae:5.95782	test-mae:6.00802
[415]	train-mae:5.95774	test-mae:6.00802
[416]	train-mae:5.95049	test-mae:6.00109
[417]	train-mae:5.95037	test-mae:6.00106
[418]	train-mae:5.9491	test-mae:5.99991
[419]	train-mae:5.94843	test-mae:5.99933
[420]	train-mae:5.9458	test-mae:5.99684
[421]	train-mae:5.94093	test-mae:5.99233
[422]	train-mae:5.93598	test-mae:5.98764
[423]	train-mae:5.93596	test-mae:5.9877
[424]	train-mae:5.93597	test-mae:5.98771
[425]	train-mae:5.93295	test-mae:5.98494
[426]	train-mae:5.9305

[604]	train-mae:5.72093	test-mae:5.79031
[605]	train-mae:5.72021	test-mae:5.78969
[606]	train-mae:5.71984	test-mae:5.78935
[607]	train-mae:5.718	test-mae:5.7877
[608]	train-mae:5.71767	test-mae:5.78743
[609]	train-mae:5.71767	test-mae:5.78745
[610]	train-mae:5.71617	test-mae:5.78613
[611]	train-mae:5.71513	test-mae:5.78514
[612]	train-mae:5.71263	test-mae:5.78269
[613]	train-mae:5.71196	test-mae:5.78214
[614]	train-mae:5.71088	test-mae:5.78117
[615]	train-mae:5.71023	test-mae:5.78056
[616]	train-mae:5.71023	test-mae:5.78057
[617]	train-mae:5.70869	test-mae:5.7791
[618]	train-mae:5.70717	test-mae:5.77765
[619]	train-mae:5.70531	test-mae:5.77582
[620]	train-mae:5.7053	test-mae:5.77584
[621]	train-mae:5.70472	test-mae:5.77536
[622]	train-mae:5.70469	test-mae:5.77534
[623]	train-mae:5.70463	test-mae:5.77535
[624]	train-mae:5.70385	test-mae:5.77467
[625]	train-mae:5.70183	test-mae:5.77271
[626]	train-mae:5.70182	test-mae:5.7727
[627]	train-mae:5.7018	test-mae:5.77274
[628]	train-mae:5.70129

[806]	train-mae:5.61277	test-mae:5.69662
[807]	train-mae:5.61235	test-mae:5.69625
[808]	train-mae:5.61208	test-mae:5.69606
[809]	train-mae:5.61113	test-mae:5.69523
[810]	train-mae:5.6111	test-mae:5.69525
[811]	train-mae:5.61015	test-mae:5.69441
[812]	train-mae:5.60971	test-mae:5.69397
[813]	train-mae:5.60905	test-mae:5.69341
[814]	train-mae:5.60905	test-mae:5.69341
[815]	train-mae:5.60904	test-mae:5.69338
[816]	train-mae:5.60854	test-mae:5.69298
[817]	train-mae:5.60853	test-mae:5.69299
[818]	train-mae:5.60794	test-mae:5.69249
[819]	train-mae:5.60793	test-mae:5.69249
[820]	train-mae:5.6069	test-mae:5.69157
[821]	train-mae:5.60687	test-mae:5.69159
[822]	train-mae:5.60631	test-mae:5.69113
[823]	train-mae:5.60592	test-mae:5.6908
[824]	train-mae:5.60561	test-mae:5.69051
[825]	train-mae:5.6056	test-mae:5.69051
[826]	train-mae:5.60468	test-mae:5.68967
[827]	train-mae:5.60438	test-mae:5.68943
[828]	train-mae:5.60437	test-mae:5.68948
[829]	train-mae:5.604	test-mae:5.68922
[830]	train-mae:5.6034

[1007]	train-mae:5.55783	test-mae:5.65476
[1008]	train-mae:5.55737	test-mae:5.65437
[1009]	train-mae:5.55691	test-mae:5.65403
[1010]	train-mae:5.55689	test-mae:5.65407
[1011]	train-mae:5.5565	test-mae:5.65379
[1012]	train-mae:5.55648	test-mae:5.65379
[1013]	train-mae:5.55642	test-mae:5.65377
[1014]	train-mae:5.55628	test-mae:5.65368
[1015]	train-mae:5.55625	test-mae:5.65369
[1016]	train-mae:5.55624	test-mae:5.65369
[1017]	train-mae:5.5561	test-mae:5.65358
[1018]	train-mae:5.55586	test-mae:5.65339
[1019]	train-mae:5.55583	test-mae:5.6534
[1020]	train-mae:5.55577	test-mae:5.6534
[1021]	train-mae:5.55577	test-mae:5.65341
[1022]	train-mae:5.55547	test-mae:5.65323
[1023]	train-mae:5.55544	test-mae:5.65321
[1024]	train-mae:5.55531	test-mae:5.65315
[1025]	train-mae:5.55526	test-mae:5.65314
[1026]	train-mae:5.55524	test-mae:5.65317
[1027]	train-mae:5.55508	test-mae:5.65311
[1028]	train-mae:5.55481	test-mae:5.6529
[1029]	train-mae:5.55458	test-mae:5.65273
[1030]	train-mae:5.55422	test-mae:5.652

[1203]	train-mae:5.53002	test-mae:5.63927
[1204]	train-mae:5.52994	test-mae:5.63923
[1205]	train-mae:5.52991	test-mae:5.63924
[1206]	train-mae:5.5296	test-mae:5.63906
[1207]	train-mae:5.52948	test-mae:5.63904
[1208]	train-mae:5.52945	test-mae:5.63904
[1209]	train-mae:5.52944	test-mae:5.63905
[1210]	train-mae:5.52943	test-mae:5.63905
[1211]	train-mae:5.52935	test-mae:5.63905
[1212]	train-mae:5.52934	test-mae:5.63905
[1213]	train-mae:5.52926	test-mae:5.63903
[1214]	train-mae:5.52918	test-mae:5.63903
[1215]	train-mae:5.52892	test-mae:5.63879
[1216]	train-mae:5.52861	test-mae:5.6385
[1217]	train-mae:5.52854	test-mae:5.63846
[1218]	train-mae:5.52842	test-mae:5.63839
[1219]	train-mae:5.5284	test-mae:5.63838
[1220]	train-mae:5.52829	test-mae:5.63839
[1221]	train-mae:5.52792	test-mae:5.63808
[1222]	train-mae:5.5277	test-mae:5.63797
[1223]	train-mae:5.52768	test-mae:5.63799
[1224]	train-mae:5.52768	test-mae:5.63797
[1225]	train-mae:5.52768	test-mae:5.63798
[1226]	train-mae:5.52765	test-mae:5.63

[1400]	train-mae:5.51166	test-mae:5.63274
[1401]	train-mae:5.51164	test-mae:5.63274
[1402]	train-mae:5.51162	test-mae:5.63277
[1403]	train-mae:5.5115	test-mae:5.63278
[1404]	train-mae:5.51131	test-mae:5.63274
[1405]	train-mae:5.51122	test-mae:5.63275
[1406]	train-mae:5.51111	test-mae:5.63275
[1407]	train-mae:5.51108	test-mae:5.63275
[1408]	train-mae:5.51097	test-mae:5.63271
[1409]	train-mae:5.51089	test-mae:5.6327
[1410]	train-mae:5.51085	test-mae:5.63272
[1411]	train-mae:5.51081	test-mae:5.63274
[1412]	train-mae:5.51072	test-mae:5.6327
[1413]	train-mae:5.51056	test-mae:5.63261
[1414]	train-mae:5.5105	test-mae:5.63263
[1415]	train-mae:5.51035	test-mae:5.63253
[1416]	train-mae:5.5103	test-mae:5.63251
[1417]	train-mae:5.51016	test-mae:5.63249
[1418]	train-mae:5.5101	test-mae:5.63249
[1419]	train-mae:5.51007	test-mae:5.63247
[1420]	train-mae:5.50997	test-mae:5.63244
[1421]	train-mae:5.5099	test-mae:5.63241
[1422]	train-mae:5.50978	test-mae:5.63243
[1423]	train-mae:5.50969	test-mae:5.63237

[1597]	train-mae:5.49841	test-mae:5.631
[1598]	train-mae:5.49826	test-mae:5.63096
[1599]	train-mae:5.49813	test-mae:5.63092
390.8243293762207


## Ordered train_test_split: 95% training

In [10]:
entries_per_day = 500
days_per_month = 30

total_months = math.floor((train.shape[0]/entries_per_day)/days_per_month)
training_ratio = 0.95
data_points = math.floor(total_months*500*30.4333*training_ratio)
print (data_points)

train_x = train.loc[:data_points][col]
y = train.loc[:data_points]['sales'] 
train_cv = train.loc[data_points+1:][col]
y_cv = train.loc[data_points+1:]['sales']

def XGB_regressor(train_X, train_y, test_X, test_y, feature_names=None, seed_val=2017, num_rounds=1600):
    param = {}
    param['objective'] = 'reg:linear'
    param['eta'] = 0.03
    #param['gamma'] = 0.1885914265710256
    param['max_depth'] = 8
    param['silent'] = 1
    param['eval_metric'] = 'mae'
    param['min_child_weight'] = 1
    param['subsample'] = 0.5947
    param['colsample_bytree'] = 0.66123
    param['seed'] = seed_val
    num_rounds = num_rounds

    plst = list(param.items())

    xgtrain = xgb.DMatrix(train_X, label=train_y)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y)
        watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=20)
    else:
        xgtest = xgb.DMatrix(test_X)
        model = xgb.train(plst, xgtrain, num_rounds)
        
    return model

start = time.time()
model = XGB_regressor(train_X = train_x, train_y = y, test_X = train_cv, test_y = y_cv)
y_test = model.predict(xgb.DMatrix(test[col]), ntree_limit = model.best_ntree_limit)
end = time.time()

sample['sales'] = y_test
sample.to_csv('xgb_ordered_test_split_95.csv', index=False)
print (end - start)

867349
[0]	train-mae:50.0803	test-mae:52.6242
Multiple eval metrics have been passed: 'test-mae' will be used for early stopping.

Will train until test-mae hasn't improved in 20 rounds.
[1]	train-mae:48.579	test-mae:51.0819
[2]	train-mae:47.1229	test-mae:49.6185
[3]	train-mae:45.7101	test-mae:48.1288
[4]	train-mae:44.3406	test-mae:46.7059
[5]	train-mae:43.0134	test-mae:45.5224
[6]	train-mae:41.726	test-mae:44.2188
[7]	train-mae:40.4798	test-mae:43.0054
[8]	train-mae:39.2721	test-mae:41.8775
[9]	train-mae:38.1055	test-mae:40.8437
[10]	train-mae:36.9806	test-mae:39.5313
[11]	train-mae:35.8902	test-mae:38.3188
[12]	train-mae:34.8453	test-mae:37.2763
[13]	train-mae:33.8326	test-mae:36.3211
[14]	train-mae:32.8661	test-mae:35.3747
[15]	train-mae:31.935	test-mae:34.432
[16]	train-mae:31.0464	test-mae:33.5362
[17]	train-mae:30.1972	test-mae:32.4986
[18]	train-mae:29.3879	test-mae:31.6236
[19]	train-mae:28.5968	test-mae:30.8297
[20]	train-mae:27.8418	test-mae:30.1059
[21]	train-mae:27.1344	tes

[201]	train-mae:7.18934	test-mae:7.71525
[202]	train-mae:7.18758	test-mae:7.71093
[203]	train-mae:7.18448	test-mae:7.70579
[204]	train-mae:7.16953	test-mae:7.69052
[205]	train-mae:7.16654	test-mae:7.68573
[206]	train-mae:7.15327	test-mae:7.67267
[207]	train-mae:7.13265	test-mae:7.649
[208]	train-mae:7.11891	test-mae:7.63502
[209]	train-mae:7.11613	test-mae:7.63142
[210]	train-mae:7.09581	test-mae:7.60881
[211]	train-mae:7.083	test-mae:7.5959
[212]	train-mae:7.08046	test-mae:7.59201
[213]	train-mae:7.07817	test-mae:7.58967
[214]	train-mae:7.07613	test-mae:7.5868
[215]	train-mae:7.07409	test-mae:7.58561
[216]	train-mae:7.06195	test-mae:7.57281
[217]	train-mae:7.04166	test-mae:7.54378
[218]	train-mae:7.03974	test-mae:7.54076
[219]	train-mae:7.03771	test-mae:7.53673
[220]	train-mae:7.01872	test-mae:7.51192
[221]	train-mae:7.01705	test-mae:7.50887
[222]	train-mae:6.99358	test-mae:7.47859
[223]	train-mae:6.99181	test-mae:7.47579
[224]	train-mae:6.97368	test-mae:7.45213
[225]	train-mae:6.972	

[402]	train-mae:5.96861	test-mae:6.30783
[403]	train-mae:5.96332	test-mae:6.30263
[404]	train-mae:5.96261	test-mae:6.30195
[405]	train-mae:5.9595	test-mae:6.29938
[406]	train-mae:5.95771	test-mae:6.29752
[407]	train-mae:5.957	test-mae:6.29661
[408]	train-mae:5.95519	test-mae:6.29544
[409]	train-mae:5.95301	test-mae:6.29269
[410]	train-mae:5.95123	test-mae:6.29064
[411]	train-mae:5.9512	test-mae:6.28988
[412]	train-mae:5.95118	test-mae:6.28925
[413]	train-mae:5.95016	test-mae:6.2888
[414]	train-mae:5.95014	test-mae:6.28942
[415]	train-mae:5.94942	test-mae:6.28872
[416]	train-mae:5.94792	test-mae:6.28732
[417]	train-mae:5.9441	test-mae:6.28334
[418]	train-mae:5.94083	test-mae:6.28023
[419]	train-mae:5.94079	test-mae:6.27983
[420]	train-mae:5.9403	test-mae:6.27944
[421]	train-mae:5.94029	test-mae:6.27883
[422]	train-mae:5.94029	test-mae:6.27888
[423]	train-mae:5.94025	test-mae:6.27946
[424]	train-mae:5.93862	test-mae:6.27779
[425]	train-mae:5.93401	test-mae:6.27308
[426]	train-mae:5.93253

[604]	train-mae:5.73257	test-mae:6.06983
[605]	train-mae:5.73125	test-mae:6.0686
[606]	train-mae:5.73027	test-mae:6.06757
[607]	train-mae:5.73022	test-mae:6.0675
[608]	train-mae:5.73017	test-mae:6.06742
[609]	train-mae:5.73017	test-mae:6.0675
[610]	train-mae:5.72978	test-mae:6.06703
[611]	train-mae:5.72978	test-mae:6.06703
[612]	train-mae:5.72979	test-mae:6.067
[613]	train-mae:5.72977	test-mae:6.06708
[614]	train-mae:5.72727	test-mae:6.06428
[615]	train-mae:5.72726	test-mae:6.0641
[616]	train-mae:5.72721	test-mae:6.06407
[617]	train-mae:5.72512	test-mae:6.06204
[618]	train-mae:5.7236	test-mae:6.06096
[619]	train-mae:5.722	test-mae:6.05981
[620]	train-mae:5.722	test-mae:6.05974
[621]	train-mae:5.71993	test-mae:6.05765
[622]	train-mae:5.71791	test-mae:6.05578
[623]	train-mae:5.71791	test-mae:6.05611
[624]	train-mae:5.71788	test-mae:6.05612
[625]	train-mae:5.71705	test-mae:6.0555
[626]	train-mae:5.71597	test-mae:6.05465
[627]	train-mae:5.71517	test-mae:6.054
[628]	train-mae:5.71444	test-m

[805]	train-mae:5.62107	test-mae:5.97511
[806]	train-mae:5.62107	test-mae:5.97506
[807]	train-mae:5.62104	test-mae:5.97506
[808]	train-mae:5.61996	test-mae:5.97385
[809]	train-mae:5.61997	test-mae:5.97394
[810]	train-mae:5.61995	test-mae:5.97389
[811]	train-mae:5.61873	test-mae:5.97285
[812]	train-mae:5.61872	test-mae:5.97287
[813]	train-mae:5.61831	test-mae:5.97261
[814]	train-mae:5.61708	test-mae:5.97116
[815]	train-mae:5.61707	test-mae:5.97115
[816]	train-mae:5.61617	test-mae:5.97058
[817]	train-mae:5.61564	test-mae:5.97018
[818]	train-mae:5.61442	test-mae:5.9688
[819]	train-mae:5.61406	test-mae:5.96883
[820]	train-mae:5.61406	test-mae:5.9686
[821]	train-mae:5.61332	test-mae:5.96781
[822]	train-mae:5.61331	test-mae:5.96784
[823]	train-mae:5.6128	test-mae:5.96757
[824]	train-mae:5.61278	test-mae:5.96766
[825]	train-mae:5.61276	test-mae:5.96769
[826]	train-mae:5.61271	test-mae:5.96775
[827]	train-mae:5.61266	test-mae:5.96771
[828]	train-mae:5.61266	test-mae:5.96781
[829]	train-mae:5.6

[1006]	train-mae:5.55993	test-mae:5.92964
[1007]	train-mae:5.55975	test-mae:5.92954
[1008]	train-mae:5.55974	test-mae:5.92959
[1009]	train-mae:5.55974	test-mae:5.92962
[1010]	train-mae:5.5594	test-mae:5.92919
[1011]	train-mae:5.55939	test-mae:5.92911
[1012]	train-mae:5.55934	test-mae:5.92932
[1013]	train-mae:5.55932	test-mae:5.92929
[1014]	train-mae:5.55902	test-mae:5.92906
[1015]	train-mae:5.559	test-mae:5.92913
[1016]	train-mae:5.55875	test-mae:5.92902
[1017]	train-mae:5.55874	test-mae:5.92901
[1018]	train-mae:5.55849	test-mae:5.92881
[1019]	train-mae:5.55813	test-mae:5.92855
[1020]	train-mae:5.55812	test-mae:5.92831
[1021]	train-mae:5.55809	test-mae:5.92846
[1022]	train-mae:5.55809	test-mae:5.92849
[1023]	train-mae:5.55806	test-mae:5.92839
[1024]	train-mae:5.55777	test-mae:5.92824
[1025]	train-mae:5.55761	test-mae:5.92795
[1026]	train-mae:5.55726	test-mae:5.92742
[1027]	train-mae:5.5571	test-mae:5.92729
[1028]	train-mae:5.55666	test-mae:5.92703
[1029]	train-mae:5.55649	test-mae:5.92